In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import torch

/Users/lucasremigio/Developer/local-llm/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print("Loading your finetuned model...")
base_model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-3n-E2B-it",
    dtype=torch.float16,
)

Loading your finetuned model...


Loading checkpoint shards: 100%|██████████| 3/3 [00:29<00:00,  9.99s/it]


In [3]:
print("Merging LoRA weights...")
model = PeftModel.from_pretrained(base_model, "outputs/lora")
model = model.merge_and_unload()

Merging LoRA weights...


/Users/lucasremigio/Developer/local-llm/venv/lib/python3.11/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


In [5]:
print("Saving merged model...")
model.save_pretrained("outputs/merged_model")

Saving merged model...


In [6]:
print("Saving tokenizer...")
tokenizer = AutoTokenizer.from_pretrained("outputs/lora")
tokenizer.save_pretrained("outputs/merged_model")


print("Done! Now convert to GGUF using llama.cpp")

Saving tokenizer...
Done! Now convert to GGUF using llama.cpp


In [7]:
# Llama.cpp needs a tokenizer.model file, so copy it from the base model
import shutil
import os

print("Copying tokenizer.model from base model...")
base_tokenizer_path = "~/.cache/huggingface/hub/models--google--gemma-3n-E2B-it/snapshots"

# Find the snapshot directory
cache_dir = os.path.expanduser(base_tokenizer_path)
if os.path.exists(cache_dir):
    # Get the latest snapshot
    snapshots = os.listdir(cache_dir)
    if snapshots:
        latest_snapshot = os.path.join(cache_dir, snapshots[0])
        tokenizer_model_src = os.path.join(latest_snapshot, "tokenizer.model")
        
        if os.path.exists(tokenizer_model_src):
            shutil.copy(tokenizer_model_src, "outputs/merged_model/tokenizer.model")
            print("✓ tokenizer.model copied")
        else:
            print("⚠️ tokenizer.model not found in cache")
else:
    print("⚠️ Cache directory not found")

Copying tokenizer.model from base model...
✓ tokenizer.model copied
